In [1]:
from vispy import gloo
import numpy as np

W, H = 1024, 1024

color_tex = gloo.Texture2D(
    shape=(H, W, 4),
    format='rgba',
    internalformat='rgba8'
)

fbo = gloo.FrameBuffer(color=color_tex)


In [2]:
VERT = """
attribute vec2 a_pos;
void main() {
    gl_Position = vec4(a_pos, 0.0, 1.0);
}
"""

FRAG = """
void main() {
    gl_FragColor = vec4(0.2, 0.3, 0.6, 1.0);
}
"""

In [3]:
program = gloo.Program(VERT, FRAG)
program['a_pos'] = gloo.VertexBuffer(
    np.array([
        [-1, -1],
        [ 1, -1],
        [-1,  1],
        [ 1,  1],
    ], dtype=np.float32)
)

with fbo:
    gloo.set_viewport(0, 0, W, H)
    gloo.clear(color=True)
    program.draw('triangle_strip')

RuntimeError: Gloo requires a Canvas to run.
If you want to use gloo without vispy.app, use a gloo.context.FakeCanvas.

In [4]:
from vispy import gloo
from vispy.gloo.context import FakeCanvas
import numpy as np

# 1. 创建 fake canvas
canvas = FakeCanvas(size=(1024, 1024))

# 2. 创建 framebuffer
color_tex = gloo.Texture2D((1024, 1024, 4), format='rgba', internalformat='rgba8')
fbo = gloo.FrameBuffer(color=color_tex)

# 3. 创建简单 shader
VERT = """
attribute vec2 a_pos;
void main() {
    gl_Position = vec4(a_pos, 0.0, 1.0);
}
"""
FRAG = """
void main() {
    gl_FragColor = vec4(0.2, 0.3, 0.6, 1.0);
}
"""
program = gloo.Program(VERT, FRAG)
program['a_pos'] = gloo.VertexBuffer(np.array([
    [-1, -1],
    [ 1, -1],
    [-1,  1],
    [ 1,  1],
], dtype=np.float32))

# 4. 渲染到 framebuffer
with fbo:
    gloo.set_viewport(0, 0, 1024, 1024)
    gloo.clear(color=True)
    program.draw('triangle_strip')

# 5. 读回数据
img = color_tex.get()  # numpy array (1024, 1024, 4)
print(img.shape, img.dtype)


TypeError: FakeCanvas.__init__() got an unexpected keyword argument 'size'

In [5]:
help(FakeCanvas)

Help on class FakeCanvas in module vispy.gloo.context:

class FakeCanvas(builtins.object)
 |  Fake canvas to allow using gloo without vispy.app
 |
 |  Instantiate this class to collect GLIR commands from gloo
 |  interactions. Call flush() in your draw event handler to execute
 |  the commands in the active contect.
 |
 |  Methods defined here:
 |
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  flush(self)
 |      Flush commands. Call this after setting to context to current.
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables (if defined)
 |
 |  __weakref__
 |      list of weak references to the object (if defined)



In [6]:
from vispy import gloo
from vispy.gloo.context import FakeCanvas
import numpy as np

W, H = 1024, 1024

# 1️⃣ 创建 fake canvas（不传 size）
# vispy 0.16
canvas = FakeCanvas()

# 2️⃣ 创建 framebuffer
color_tex = gloo.Texture2D((H, W, 4), format='rgba', internalformat='rgba8')
fbo = gloo.FrameBuffer(color=color_tex)

# 3️⃣ 创建 shader
VERT = """
attribute vec2 a_pos;
void main() {
    gl_Position = vec4(a_pos, 0.0, 1.0);
}
"""

FRAG = """
void main() {
    gl_FragColor = vec4(0.2, 0.3, 0.6, 1.0);
}
"""

program = gloo.Program(VERT, FRAG)
program['a_pos'] = gloo.VertexBuffer(np.array([
    [-1, -1],
    [ 1, -1],
    [-1,  1],
    [ 1,  1],
], dtype=np.float32))

# 4️⃣ 渲染到 framebuffer
with fbo:
    gloo.set_viewport(0, 0, W, H)
    gloo.clear(color=True)
    program.draw('triangle_strip')

# 5️⃣ 读取数据
img = color_tex.get()  # numpy array (H, W, 4)
print(img.shape, img.dtype)


IndexError: list index out of range

In [7]:
# panda_vispy_offscreen.py
from panda3d.core import *
from direct.showbase.ShowBase import ShowBase
from vispy import gloo
import numpy as np

# ----------------------------
# 1️⃣ Panda3D 主窗口
# ----------------------------
class MyApp(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)
        
        W, H = 512, 512  # 离屏渲染尺寸

        # ----------------------------
        # 2️⃣ 创建 VisPy FBO
        # ----------------------------
        # 注意：不创建 FakeCanvas！
        # 直接在当前 Panda3D context 下使用 gloo
        color_tex = gloo.Texture2D((H, W, 4), format='rgba', internalformat='rgba8')
        fbo = gloo.FrameBuffer(color=color_tex)

        # ----------------------------
        # 3️⃣ 创建 shader
        # ----------------------------
        VERT = """
        attribute vec2 a_pos;
        void main() {
            gl_Position = vec4(a_pos, 0.0, 1.0);
        }
        """

        FRAG = """
        void main() {
            gl_FragColor = vec4(0.2, 0.6, 0.3, 1.0);  // 绿色星空色
        }
        """

        program = gloo.Program(VERT, FRAG)
        program['a_pos'] = gloo.VertexBuffer(np.array([
            [-1, -1],
            [ 1, -1],
            [-1,  1],
            [ 1,  1],
        ], dtype=np.float32))

        # ----------------------------
        # 4️⃣ 离屏渲染
        # ----------------------------
        # 必须保证 Panda3D 的 GL context active
        # render_frame() 在 Panda3D context 下执行
        def render_frame():
            with fbo:
                gloo.set_viewport(0, 0, W, H)
                gloo.clear(color=True)
                program.draw('triangle_strip')
            # 从 FBO 读回 numpy
            img = color_tex.get()  # (H,W,4)
            return img

        self.star_img = render_frame()

        # ----------------------------
        # 5️⃣ 创建 Panda3D 纹理并显示
        # ----------------------------
        tex = Texture()
        tex.setup_2d_texture(W, H, Texture.T_unsigned_byte, Texture.F_rgba)
        tex.set_ram_image(self.star_img.astype(np.uint8).tobytes())

        cm = CardMaker('bg')
        cm.set_frame(-1, 1, -1, 1)
        card = self.render2d.attach_new_node(cm.generate())
        card.set_texture(tex)
        card.set_transparency(True)

        print("离屏星空生成完成")

# ----------------------------
# 6️⃣ 启动
# ----------------------------
if __name__ == "__main__":
    app = MyApp()
    app.run()


Known pipe types:
  glxGraphicsPipe
(1 aux display modules not yet loaded.)
AL lib: (EE) GetLoadedHrtf: Invalid header in /usr/share/openal/hrtf/Default HRTF.mhr: "MinPHR03"
AL lib: (EE) GetLoadedHrtf: Failed to load /usr/share/openal/hrtf/Default HRTF.mhr


IndexError: list index out of range

In [1]:
# panda3d_vispy_task.py
from direct.showbase.ShowBase import ShowBase
from panda3d.core import CardMaker, Texture
from vispy import gloo
import numpy as np

# ----------------------------
# 配置离屏渲染大小
# ----------------------------
W, H = 512, 512

# ----------------------------
# 1️⃣ 创建离屏 FBO 和 Shader
# ----------------------------
color_tex = gloo.Texture2D((H, W, 4), format='rgba', internalformat='rgba8')
fbo = gloo.FrameBuffer(color=color_tex)

VERT = """
attribute vec2 a_pos;
void main() {
    gl_Position = vec4(a_pos, 0.0, 1.0);
}
"""

FRAG = """
void main() {
    gl_FragColor = vec4(0.2, 0.6, 0.3, 1.0);  // 绿色星空色
}
"""

program = gloo.Program(VERT, FRAG)
program['a_pos'] = gloo.VertexBuffer(np.array([
    [-1, -1],
    [ 1, -1],
    [-1,  1],
    [ 1,  1],
], dtype=np.float32))


# ----------------------------
# 2️⃣ Panda3D 主程序
# ----------------------------
class MyApp(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)

        # ----------------------------
        # 2️⃣ 创建 Panda3D 背景卡片
        # ----------------------------
        self.tex = Texture()
        self.tex.setup_2d_texture(W, H, Texture.T_unsigned_byte, Texture.F_rgba)

        cm = CardMaker('bg')
        cm.set_frame(-1, 1, -1, 1)
        card = self.render2d.attach_new_node(cm.generate())
        card.set_texture(self.tex)
        card.set_transparency(True)

        # ----------------------------
        # 3️⃣ 添加 task，在 GL context 已激活时渲染 FBO
        # ----------------------------
        self.taskMgr.add(self.render_star_task, "render_star_task")

    # ----------------------------
    # 4️⃣ task：渲染 VisPy FBO 并更新 Panda3D 纹理
    # ----------------------------
    def render_star_task(self, task):
        # 确保 Panda3D 的 GL context active
        self.win.getGsg().makeCurrent()

        # VisPy 离屏渲染
        with fbo:
            gloo.set_viewport(0, 0, W, H)
            gloo.clear(color=True)
            program.draw('triangle_strip')

        # 读回数据，上传 Panda3D 纹理
        img = color_tex.get()  # numpy array (H,W,4)
        self.tex.set_ram_image(img.astype(np.uint8).tobytes())

        return task.cont  # 持续更新

# ----------------------------
# 5️⃣ 启动程序
# ----------------------------
if __name__ == "__main__":
    app = MyApp()
    app.run()


Known pipe types:
  glxGraphicsPipe
(1 aux display modules not yet loaded.)
AL lib: (EE) GetLoadedHrtf: Invalid header in /usr/share/openal/hrtf/Default HRTF.mhr: "MinPHR03"
AL lib: (EE) GetLoadedHrtf: Failed to load /usr/share/openal/hrtf/Default HRTF.mhr


AttributeError: 'panda3d.core.GraphicsStateGuardian' object has no attribute 'makeCurrent'

:task(error): Exception occurred in PythonTask render_star_task


AttributeError: 'panda3d.core.GraphicsStateGuardian' object has no attribute 'makeCurrent'

In [1]:
# panda3d_vispy_task_fixed.py
from direct.showbase.ShowBase import ShowBase
from panda3d.core import CardMaker, Texture
from vispy import gloo
import numpy as np

# ----------------------------
# 配置离屏渲染大小
# ----------------------------
W, H = 512, 512

# ----------------------------
# 1️⃣ 创建离屏 FBO 和 Shader
# ----------------------------
color_tex = gloo.Texture2D((H, W, 4), format='rgba', internalformat='rgba8')
fbo = gloo.FrameBuffer(color=color_tex)

VERT = """
attribute vec2 a_pos;
void main() {
    gl_Position = vec4(a_pos, 0.0, 1.0);
}
"""

FRAG = """
void main() {
    gl_FragColor = vec4(0.2, 0.6, 0.3, 1.0);  // 绿色星空色
}
"""

program = gloo.Program(VERT, FRAG)
program['a_pos'] = gloo.VertexBuffer(np.array([
    [-1, -1],
    [ 1, -1],
    [-1,  1],
    [ 1,  1],
], dtype=np.float32))


# ----------------------------
# 2️⃣ Panda3D 主程序
# ----------------------------
class MyApp(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)

        # ----------------------------
        # 2️⃣ 创建 Panda3D 背景卡片
        # ----------------------------
        self.tex = Texture()
        self.tex.setup_2d_texture(W, H, Texture.T_unsigned_byte, Texture.F_rgba)

        cm = CardMaker('bg')
        cm.set_frame(-1, 1, -1, 1)
        card = self.render2d.attach_new_node(cm.generate())
        card.set_texture(self.tex)
        card.set_transparency(True)

        # ----------------------------
        # 3️⃣ 添加 task，在 GL context 已激活时渲染 FBO
        # ----------------------------
        self.taskMgr.add(self.render_star_task, "render_star_task")

    # ----------------------------
    # 4️⃣ task：渲染 VisPy FBO 并更新 Panda3D 纹理
    # ----------------------------
    def render_star_task(self, task):
        # 不需要 makeCurrent，task 执行时 context 已经 active
        with fbo:
            gloo.set_viewport(0, 0, W, H)
            gloo.clear(color=True)
            program.draw('triangle_strip')

        # 读回数据，上传 Panda3D 纹理
        img = color_tex.get()  # numpy array (H,W,4)
        self.tex.set_ram_image(img.astype(np.uint8).tobytes())

        return task.cont  # 持续更新

# ----------------------------
# 5️⃣ 启动程序
# ----------------------------
if __name__ == "__main__":
    app = MyApp()
    app.run()


Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


RuntimeError: Gloo requires a Canvas to run.
If you want to use gloo without vispy.app, use a gloo.context.FakeCanvas.

:task(error): Exception occurred in PythonTask render_star_task


RuntimeError: Gloo requires a Canvas to run.
If you want to use gloo without vispy.app, use a gloo.context.FakeCanvas.

In [1]:
# asdfasdf
import numpy as np
from vispy import app, gloo
from vispy import app
app.use_app('glfw')   # 或 'pyqt5'
# ----------------------
# Shader
# ----------------------
VERT = """
attribute vec2 a_pos;
void main() {
    gl_Position = vec4(a_pos, 0.0, 1.0);
}
"""

FRAG = """
void main() {
    gl_FragColor = vec4(0.2, 0.4, 0.8, 1.0);
}
"""

# ----------------------
# Canvas
# ----------------------
class SimpleCanvas(app.Canvas):
    def __init__(self):
        app.Canvas.__init__(self, title='Simple gloo demo',
                            size=(600, 600), show=True)

        self.program = gloo.Program(VERT, FRAG)

        self.program['a_pos'] = gloo.VertexBuffer(np.array([
            [-1, -1],
            [ 1, -1],
            [-1,  1],
            [ 1,  1],
        ], dtype=np.float32))

    def on_draw(self, event):
        gloo.clear(color=True)
        self.program.draw('triangle_strip')

# ----------------------
# Run
# ----------------------
if __name__ == '__main__':
    c = SimpleCanvas()
    app.run()


  File "/mnt/D/packages/miniconda3/envs/game/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/mnt/D/packages/miniconda3/envs/game/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/mnt/D/packages/miniconda3/envs/game/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/mnt/D/packages/miniconda3/envs/game/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/mnt/D/packages/miniconda3/envs/game/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  File "/mnt/D/packages/miniconda3/envs/game/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "/mnt/D/packages/miniconda3/envs/game/lib/python3.12/asyncio/base_events.py", line 645, in run_forever
    self._run

In [1]:
import numpy as np
from panda3d.core import Texture, CardMaker, NodePath, LVecBase4
from direct.showbase.ShowBase import ShowBase
# from vispy import gloo, gl
from vispy import gloo
from vispy.gloo import gl


# ----------------------------
# 1️⃣ VisPy Shader 和 FBO 初始化
# ----------------------------
W, H = 512, 512

VERT_SHADER = """
attribute vec2 a_pos;
void main() {
    gl_Position = vec4(a_pos, 0.0, 1.0);
}
"""

FRAG_SHADER = """
void main() {
    gl_FragColor = vec4(0.2, 0.3, 0.6, 1.0);
}
"""

# 顶点数据
vertex_data = np.array([
    [-1, -1],
    [ 1, -1],
    [-1,  1],
    [ 1,  1],
], dtype=np.float32)

# ----------------------------
# 2️⃣ Panda3D App
# ----------------------------
class MyApp(ShowBase):
    def __init__(self):
        super().__init__()

        # 创建 Panda3D 纹理
        self.tex = Texture()
        self.tex.setup_2d_texture(W, H, Texture.T_unsigned_byte, Texture.F_rgba8)

        # 创建一个屏幕卡片显示纹理
        cm = CardMaker("card")
        cm.set_frame_fullscreen_quad()
        card = NodePath(cm.generate())
        card.set_texture(self.tex)
        card.reparent_to(self.render2d)

        # 初始化 FBO 和 Shader
        self.program = gloo.Program(VERT_SHADER, FRAG_SHADER)
        self.program['a_pos'] = gloo.VertexBuffer(vertex_data)

        self.color_tex = gloo.Texture2D((H, W, 4), format='rgba', internalformat='rgba8')
        self.fbo = gloo.FrameBuffer(color=self.color_tex)

        # 添加任务，每帧更新
        self.taskMgr.add(self.render_star_task, "render_star")

    def render_star_task(self, task):
        # Panda3D GL context 已经 active
        # VisPy FBO 渲染
        with self.fbo:
            gl.glViewport(0, 0, W, H)
            gloo.clear(color=True)
            self.program.draw('triangle_strip')

        # 读回数据，转成 0-255 uint8
        # old interface; new interface does not have get
        # img = (self.color_tex.get() * 255).astype(np.uint8)
        

        # 更新 Panda3D 纹理
        # self.tex.set_ram_image(img.tobytes())
        img = gloo.read_pixels((0, 0, W, H), alpha=True)  # 返回 (H, W, 4) float32 [0,1]
        img_uint8 = (img * 255).astype(np.uint8)
        self.tex.set_ram_image(img_uint8.tobytes())

        return task.cont

# ----------------------------
# 3️⃣ 启动
# ----------------------------
if __name__ == "__main__":
    app = MyApp()
    app.run()


Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


RuntimeError: Gloo requires a Canvas to run.
If you want to use gloo without vispy.app, use a gloo.context.FakeCanvas.

:task(error): Exception occurred in PythonTask render_star


RuntimeError: Gloo requires a Canvas to run.
If you want to use gloo without vispy.app, use a gloo.context.FakeCanvas.

In [2]:
import numpy as np
from vispy import gloo
from vispy.gloo import gl

# ----------------------------
# 1️⃣ FBO 尺寸与 Shader
# ----------------------------
W, H = 512, 512

VERT_SHADER = """
attribute vec2 a_pos;
void main() {
    gl_Position = vec4(a_pos, 0.0, 1.0);
}
"""

FRAG_SHADER = """
void main() {
    gl_FragColor = vec4(0.2, 0.3, 0.6, 1.0);
}
"""

# 顶点数据
vertex_data = np.array([
    [-1, -1],
    [ 1, -1],
    [-1,  1],
    [ 1,  1],
], dtype=np.float32)

# ----------------------------
# 2️⃣ 创建离屏渲染 FBO
# ----------------------------
# 创建颜色纹理
color_tex = gloo.Texture2D((H, W, 4), format='rgba', internalformat='rgba8')

# 创建 Framebuffer
fbo = gloo.FrameBuffer(color=color_tex)

# 创建 Shader Program
program = gloo.Program(VERT_SHADER, FRAG_SHADER)
program['a_pos'] = gloo.VertexBuffer(vertex_data)

# ----------------------------
# 3️⃣ 渲染到 FBO
# ----------------------------
with fbo:
    # 设置视口
    gl.glViewport(0, 0, W, H)
    # 清空颜色缓冲
    gloo.clear(color=True)
    # 绘制四边形
    program.draw('triangle_strip')

# ----------------------------
# 4️⃣ 读取数据
# ----------------------------
# read_pixels 返回 (H, W, 4) float32 数组，值在 [0,1]
img = gloo.read_pixels((0, 0, W, H), alpha=True)
# 转为 0-255 uint8
img_uint8 = (img * 255).astype(np.uint8)

print("离屏渲染完成，图像形状:", img_uint8.shape, "dtype:", img_uint8.dtype)


RuntimeError: Gloo requires a Canvas to run.
If you want to use gloo without vispy.app, use a gloo.context.FakeCanvas.